In [2]:
%%bash

which python

# Get Data
sh ../netflix/input/download_data.sh
ls

/home/kiran/.miniconda3/envs/dgraphpandas/bin/python

Archive:  netflix-shows.zip
  inflating: netflix_titles.csv      
NetflixSample.ipynb
PlanetSample.ipynb
netflix_titles.csv


100%|██████████| 1.18M/1.18M [00:00<00:00, 4.68MB/s]


In [3]:
%%bash

# Start DGraph (remember to shut them down at end)
docker-compose -f ../../docker-compose.yml up -d
docker container ls

CONTAINER ID        IMAGE                  COMMAND                  CREATED             STATUS                  PORTS                                                                NAMES
4afa67c17f8f        dgraph/dgraph:latest   "dgraph-ratel"           1 second ago        Up Less than a second   8080/tcp, 0.0.0.0:8000->8000/tcp, 9080/tcp                           dgraphpandas_ratel_1
fa0b28998453        dgraph/dgraph:latest   "dgraph alpha --my=a…"   1 second ago        Up Less than a second   0.0.0.0:8080->8080/tcp, 0.0.0.0:9080->9080/tcp                       dgraphpandas_alpha_1
b0c1a1610a66        dgraph/dgraph:latest   "dgraph zero --my=ze…"   1 second ago        Up Less than a second   0.0.0.0:5080->5080/tcp, 8080/tcp, 0.0.0.0:6080->6080/tcp, 9080/tcp   dgraphpandas_zero_1


Creating network "dgraphpandas_default" with the default driver
Creating dgraphpandas_zero_1 ... 
Creating dgraphpandas_alpha_1 ... 
Creating dgraphpandas_ratel_1 ... 
Creating dgraphpandas_ratel_1 ... done
Creating dgraphpandas_alpha_1 ... done
Creating dgraphpandas_zero_1  ... done


In [4]:
%%bash

python -m pip install dgraphpandas

python ../netflix/input/download_cleanup.py
ls

Generating title types
Generating directors
Generating cast
Generating rating
Generating genre
NetflixSample.ipynb
PlanetSample.ipynb
cast.csv
directors.csv
genre.csv
netflix_titles.csv
rating.csv
show_types.csv


In [5]:
%%bash

# Generate Upserts

python -m dgraphpandas -c ../netflix/dgraphpandas.json -ck director -f directors.csv --console
python -m dgraphpandas -c ../netflix/dgraphpandas.json -ck title -f netflix_titles.csv --console

Intrinsic:
                        subject    predicate             object         type
0      director_JorgeMichelGrau   identifier  Jorge Michel Grau  <xs:string>
1          director_GilbertChan   identifier       Gilbert Chan  <xs:string>
2           director_ShaneAcker   identifier        Shane Acker  <xs:string>
3        director_RobertLuketic   identifier     Robert Luketic  <xs:string>
4           director_SerdarAkar   identifier        Serdar Akar  <xs:string>
...                         ...          ...                ...          ...
6105  director_RodrigoGuardiola  dgraph.type           director  <xs:string>
6106  director_GabrielCruzRivas  dgraph.type           director  <xs:string>
6111       director_PeterHewitt  dgraph.type           director  <xs:string>
6112        director_JosefFares  dgraph.type           director  <xs:string>
6113        director_MozezSingh  dgraph.type           director  <xs:string>

[10593 rows x 4 columns]
Edges:
Empty DataFrame
Columns: [subjec

2021-04-04 16:39:16 DESKTOP-4HNUNO0 __main__[4076] DEBUG Global Config 
 {'files': {'cast': {'pre_rename': {'cast': 'identifier'},
                    'subject_fields': ['id']},
           'director': {'pre_rename': {'director': 'identifier'},
                        'subject_fields': ['id']},
           'genre': {'pre_rename': {'genre': 'identifier'},
                     'subject_fields': ['id']},
           'rating': {'pre_rename': {'rating': 'identifier'},
                      'subject_fields': ['id']},
           'show_types': {'pre_rename': {'type': 'identifier'},
                          'subject_fields': ['id']},
           'title': {'csv_edges': ['director', 'cast', 'genre', 'country'],
                     'edge_fields': ['type',
                                     'director',
                                     'cast',
                                     'country',
                                     'rating',
                                     'genre'],
            

In [7]:
!ls *.gz

directors_edges.gz	netflix_titles_edges.gz
directors_intrinsic.gz	netflix_titles_intrinsic.gz


In [14]:
# Ensure XID predicate exists for upsert operations
# If you get auth issue, then take a look at the whitelist ip in the docker-compose file
!curl -sX POST localhost:8080/alter -d 'xid: string @index(exact) .' | jq .

{
  "data": {
    "code": "Success",
    "message": "Done"
  }
}


In [15]:
%%bash

#Apply Upserts to Dgraph
# Omitting the rest of the files for breveity, but it's the same following steps for all of them
dgraph live --files directors_intrinsic.gz --upsertPredicate xid --xidmap xidmap --format rdf --batch 500
dgraph live --files directors_edges.gz --upsertPredicate xid --xidmap xidmap --format rdf --batch 500
dgraph live --files netflix_titles_intrinsic.gz --upsertPredicate xid --xidmap xidmap --format rdf --batch 500
dgraph live --files netflix_titles_edges.gz --upsertPredicate xid --xidmap xidmap --format rdf --batch 500

[Decoder]: Using assembly version of decoder
Page Size: 4096

Running transaction with dgraph endpoint: 127.0.0.1:9080
Found 1 data file(s) to process
Processing data file "directors_intrinsic.gz"
Number of TXs run            : 22
Number of N-Quads processed  : 10593
Time spent                   : 514.0192ms
N-Quads processed per second : 10593
[Decoder]: Using assembly version of decoder
Page Size: 4096

Running transaction with dgraph endpoint: 127.0.0.1:9080
Found 1 data file(s) to process
Processing data file "directors_edges.gz"
Number of TXs run            : 0
Number of N-Quads processed  : 0
Time spent                   : 2.7456ms
N-Quads processed per second : 0
[Decoder]: Using assembly version of decoder
Page Size: 4096

Running transaction with dgraph endpoint: 127.0.0.1:9080
Found 1 data file(s) to process
Processing data file "netflix_titles_intrinsic.gz"
Number of TXs run            : 94
Number of N-Quads processed  : 46712
Time spent                   : 2.0965622s
N-Quad

I0404 16:43:45.990922    4539 init.go:107] 

Dgraph version   : v20.11.2
Dgraph codename  : tchalla-2
Dgraph SHA-256   : 0153cb8d3941ad5ad107e395b347e8d930a0b4ead6f4524521f7a525a9699167
Commit SHA-1     : 94f3a0430
Commit timestamp : 2021-02-23 13:07:17 +0530
Branch           : HEAD
Go version       : go1.15.5
jemalloc enabled : true

For Dgraph official documentation, visit https://dgraph.io/docs/.
For discussions about Dgraph     , visit https://discuss.dgraph.io.

Licensed variously under the Apache Public License 2.0 and Dgraph Community License.
Copyright 2015-2020 Dgraph Labs, Inc.


badger 2021/04/04 16:43:46 INFO: All 0 tables opened in 0s
badger 2021/04/04 16:43:46 INFO: Discard stats nextEmptySlot: 0
badger 2021/04/04 16:43:46 INFO: Set nextTxnTs to 0
I0404 16:43:46.021243    4539 xidmap.go:128] Loaded up 0 xid to uid mappings
I0404 16:43:46.535335    4539 xidmap.go:288] Writing xid map to DB
I0404 16:43:46.535366    4539 xidmap.go:290] Finished writing xid map to DB
badger 2

In [16]:
%%bash
echo Applying Reverse Edges
curl -sX POST localhost:8080/alter -d 'cast: [uid] @reverse .' | jq .
curl -sX POST localhost:8080/alter -d 'genre: [uid] @reverse .' | jq .
curl -sX POST localhost:8080/alter -d 'director: [uid] @reverse .' | jq .
curl -sX POST localhost:8080/alter -d 'rating: [uid] @reverse .' | jq .
curl -sX POST localhost:8080/alter -d 'description: string @index(term) .' | jq .

Applying Reverse Edges
{
  "data": {
    "code": "Success",
    "message": "Done"
  }
}
{
  "data": {
    "code": "Success",
    "message": "Done"
  }
}
{
  "data": {
    "code": "Success",
    "message": "Done"
  }
}
{
  "data": {
    "code": "Success",
    "message": "Done"
  }
}
{
  "data": {
    "code": "Success",
    "message": "Done"
  }
}


In [17]:
%%bash

# Example Query
curl -sX POST localhost:8080/query -H 'Content-Type: application/dql' -d '
{
    directors(func: type(director), first: 10)
    {
        identifier
        titles: ~director(first: 5) @filter(type(title))
        {
            title
            date_added
            release_year
        }
    }
}
' | jq .

{
  "data": {
    "directors": [
      {
        "identifier": "Alexis Morante",
        "titles": [
          {
            "title": "Alejandro Sanz: What I Was Is What I Am",
            "date_added": "2018-08-18T00:00:00Z",
            "release_year": 2018
          },
          {
            "title": "Camarón: The Film",
            "date_added": "2018-07-18T00:00:00Z",
            "release_year": 2018
          }
        ]
      },
      {
        "identifier": "Ishaan Trivedi",
        "titles": [
          {
            "title": "A Scandall",
            "date_added": "2017-10-15T00:00:00Z",
            "release_year": 2016
          }
        ]
      },
      {
        "identifier": "Brendon Marotta",
        "titles": [
          {
            "title": "American Circumcision",
            "date_added": "2018-12-16T00:00:00Z",
            "release_year": 2017
          }
        ]
      },
      {
        "identifier": "Alfonso Serrano Maturino",
        "titles": [
          {

In [18]:
%%bash

# Stop DGraph
docker-compose -f ../../docker-compose.yml down

Stopping dgraphpandas_ratel_1 ... 
Stopping dgraphpandas_alpha_1 ... 
Stopping dgraphpandas_zero_1  ... 
Stopping dgraphpandas_ratel_1 ... done
Stopping dgraphpandas_alpha_1 ... done
Stopping dgraphpandas_zero_1  ... done
Removing dgraphpandas_ratel_1 ... 
Removing dgraphpandas_alpha_1 ... 
Removing dgraphpandas_zero_1  ... 
Removing dgraphpandas_ratel_1 ... done
Removing dgraphpandas_alpha_1 ... done
Removing dgraphpandas_zero_1  ... done
Removing network dgraphpandas_default


In [19]:
%%bash

# Clean up
rm *.csv
rm *.gz
rm -rf t
rm -rf xidmap